In [ ]:
import pandas as pd

df = pd.DataFrame().from_dict({'X': [0,1,2,28,18,30], 'Y': [0,0,0,1,1,1]})
df

## Let's visulize it

In [ ]:
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, CustomJS, Slider, Div
from bokeh.plotting import figure, output_notebook, show
from bokeh.themes import built_in_themes
from bokeh.io import curdoc

output_notebook()
curdoc().theme = 'night_sky'

source = ColumnDataSource(df)

plot = figure(plot_width=400, plot_height=400)
plot.circle(x='X', y='Y', source=source, size=10, color="green", alpha=0.5)

show(plot)

# Slider Game (again?)

In [ ]:
import math
def sigmiod_func(df):
    print(df)
    return 1/1+math.exp(-1*(df.b1*df.X+df.b0))

b1 = 0.425
b0 = 0.785

df = pd.DataFrame().from_dict({'X': [0,1,2,28,18,30], 'Y': [0,0,0,1,1,1]})
df['b1'] = b1
df['b0'] = b0

df['Y pred'] = df.apply(sigmiod_func, axis=1)
df

In [ ]:
import math
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, CustomJS, Slider, Div
from bokeh.plotting import figure, output_notebook, show
from bokeh.themes import built_in_themes
from bokeh.io import curdoc

output_notebook()
curdoc().theme = 'night_sky'

def sigmiod_func(df):
    return 1/1+math.exp(-1*(df.b1*df.X+df.b0))

b1 = 0.425
b0 = 0.785

df = pd.DataFrame().from_dict({'X': [0,1,2,28,18,30], 'Y': [0,0,0,1,1,1]})
df['b1'] = b1
df['b0'] = b0
df['Y pred'] = df.apply(sigmiod_func)

source = ColumnDataSource(df)

plot = figure(plot_width=400, plot_height=400, x_range=[0,6], y_range=[0,6])
plot.line(x='X', y='Y pred', source=source, line_width=3, line_alpha=0.6)
plot.circle(x='X', y='Y', source=source, size=10, color="green", alpha=0.5)

callback_b = CustomJS(args=dict(source=source), code="""
        var data = source.data;
        var b_val = cb_obj.value
        var b = data['b']
        var A = data['A']
        var x = data['X']
        var y = data['Y pred']
        for (var i = 0; i < x.length; i++) {
            b[i] = b_val
            y[i] = b[i] * x[i] + A[i]
        }
        source.change.emit();
    """)

callback_A = CustomJS(args=dict(source=source), code="""
        var data = source.data;
        var A_val = cb_obj.value
        var b = data['b']
        var A = data['A']
        var x = data['X']
        var y = data['Y pred']
        for (var i = 0; i < x.length; i++) {
            A[i] = A_val
            y[i] = b[i] * x[i] + A[i]
        }
        source.change.emit();
    """)

div = Div(text="Sum of error sq: "+ str(sum_error_eq(source.data['X'],source.data['Y'], b, A)))

change_text = CustomJS(args=dict(dev=div, source=source), code="""
                var data = source.data;
                var y_pred = data['Y pred'];
                var y = data['Y'];
                var result = 0;
                for (var i = 0; i < y.length; i++) {
                    var diff = y_pred[i] - y[i]
                    result = result + diff * diff
                }
                dev.text = "Sum of error sq: " + result;
             """       
    )

slider_b = Slider(start=0.1, end=4, value=b, step=.1, title="value of b")
slider_b.js_on_change('value', callback_b, change_text)
slider_A = Slider(start=0.1, end=4, value=A, step=.1, title="value of A")
slider_A.js_on_change('value', callback_A, change_text)

layout = column(slider_b, slider_A, plot, div)

show(layout)

### Predicting with scikit learn
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

In [ ]:
import numpy as np

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0)
clf.fit(df["X"].values.reshape(-1, 1), df["Y"].values)


In [ ]:
clf.predict(np.array(12).reshape(-1, 1))

In [ ]:
clf.predict_proba(np.array(10).reshape(-1, 1))